In [160]:
import pyomo.environ as pyo

# Datos de entrada
datos = {
    "demanda": 600,
    "centros_acopio": [
        {"id": 0, "stock": 100, "produccion_potencial": 150, "precio": 10000, "tiempo_alistamiento": 1},
        {"id": 1, "stock": 200, "produccion_potencial": 200, "precio": 15000, "tiempo_alistamiento": 2},
        {"id": 2, "stock": 150, "produccion_potencial": 180, "precio": 12000, "tiempo_alistamiento": 1.5},
        {"id": 3, "stock": 120, "produccion_potencial": 170, "precio": 11000, "tiempo_alistamiento": 1.2},
        {"id": 4, "stock": 180, "produccion_potencial": 190, "precio": 14000, "tiempo_alistamiento": 2.1},
        {"id": 5, "stock": 160, "produccion_potencial": 160, "precio": 13000, "tiempo_alistamiento": 1.8},
        {"id": 6, "stock": 110, "produccion_potencial": 140, "precio": 16000, "tiempo_alistamiento": 1.7},
        {"id": 7, "stock": 140, "produccion_potencial": 155, "precio": 17000, "tiempo_alistamiento": 2.2},
        {"id": 8, "stock": 130, "produccion_potencial": 175, "precio": 18000, "tiempo_alistamiento": 1.9}
    ],
    "costos_transporte": [
        [0, 5, 7, 6, 8, 7, 9, 10, 11],
        [5, 0, 3, 4, 5, 6, 7, 8, 9],
        [7, 3, 0, 5, 4, 6, 8, 7, 10],
        [6, 4, 5, 0, 6, 7, 8, 9, 11],
        [8, 5, 4, 6, 0, 8, 7, 9, 10],
        [7, 6, 6, 7, 8, 0, 5, 8, 9],
        [9, 7, 8, 8, 7, 5, 0, 6, 7],
        [10, 8, 7, 9, 9, 8, 6, 0, 5],
        [11, 9, 10, 11, 10, 9, 7, 5, 0]
    ],
    "tiempos_transporte": [
        [0, 1, 2, 1.5, 2, 1.8, 2.1, 2.2, 2.5],
        [1, 0, 1, 1.2, 1.5, 1.3, 1.8, 1.9, 2.1],
        [2, 1, 0, 1.3, 1.4, 1.6, 2, 1.7, 2],
        [1.5, 1.2, 1.3, 0, 1.6, 1.5, 1.8, 2, 2.2],
        [2, 1.5, 1.4, 1.6, 0, 1.7, 2, 1.8, 2.3],
        [1.8, 1.3, 1.6, 1.5, 1.7, 0, 1.8, 1.9, 2],
        [2.1, 1.8, 2, 1.8, 2, 1.8, 0, 1.5, 1.8],
        [2.2, 1.9, 1.7, 2, 1.8, 1.9, 1.5, 0, 1.6],
        [2.5, 2.1, 2, 2.2, 2.3, 2, 1.8, 1.6, 0]
    ],
    "parametros_modelo": {
        "costo_tiempo": 2,
        "tiempo_maximo_definido": 5
    }
}

# Crear el modelo de optimización
model = pyo.ConcreteModel()

# Conjuntos
model.I = pyo.RangeSet(len(datos['centros_acopio']) - 1)  # Centros de acopio

# Parámetros
model.Demanda = pyo.Param(initialize=datos['demanda'])
model.Stock = pyo.Param(model.I, initialize={i: datos['centros_acopio'][i]['stock'] for i in model.I})
model.Pproduccion = pyo.Param(model.I, initialize={i: datos['centros_acopio'][i]['produccion_potencial'] for i in model.I})
model.Precio = pyo.Param(model.I, initialize={i: datos['centros_acopio'][i]['precio'] for i in model.I})
model.cTransp = pyo.Param(model.I, model.I, initialize={(i, j): datos['costos_transporte'][i][j] for i in model.I for j in model.I})
model.tTransp = pyo.Param(model.I, model.I, initialize={(i, j): datos['tiempos_transporte'][i][j] for i in model.I for j in model.I})
model.tAlistam = pyo.Param(model.I, initialize={i: datos['centros_acopio'][i]['tiempo_alistamiento'] for i in model.I})
model.cTiempo = pyo.Param(initialize=datos['parametros_modelo']['costo_tiempo'])
model.tMax = pyo.Param(initialize=datos['parametros_modelo']['tiempo_maximo_definido'])

# Selección automática del centro principal basado en el costo más bajo
centro_principal_id = min(model.I, key=lambda i: datos['centros_acopio'][i]['precio'])
model.CentroPrincipal = pyo.Param(initialize=centro_principal_id)

# Variables
model.X = pyo.Var(model.I, within=pyo.NonNegativeReals)  # Cantidad despachada desde cada centro
model.Y = pyo.Var(model.I, model.I, within=pyo.NonNegativeReals)  # Cantidad transportada entre centros

# Restricciones
def demanda_satisfecha_rule(model):
    return sum(model.X[i] for i in model.I) == model.Demanda
model.DemandaSatisfecha = pyo.Constraint(rule=demanda_satisfecha_rule)

def stock_disponible_rule(model, i):
    return model.X[i] <= model.Stock[i]
model.StockDisponible = pyo.Constraint(model.I, rule=stock_disponible_rule)

def produccion_potencial_rule(model, i):
    return model.X[i] <= model.Pproduccion[i]
model.ProduccionPotencial = pyo.Constraint(model.I, rule=produccion_potencial_rule)

def tiempo_max_rule(model, i):
    return model.tAlistam[i] + sum(model.tTransp[i, j] * model.Y[i, j] for j in model.I) <= model.tMax
model.TiempoMax = pyo.Constraint(model.I, rule=tiempo_max_rule)

def balance_transport_rule(model, i):
    return sum(model.Y[j, i] for j in model.I) == model.X[i] if i != model.CentroPrincipal else pyo.Constraint.Skip
model.BalanceTransport = pyo.Constraint(model.I, rule=balance_transport_rule)

def centro_principal_rule(model):
    return sum(model.Y[i, model.CentroPrincipal] for i in model.I) == model.X[model.CentroPrincipal]
model.CentroPrincipalManejo = pyo.Constraint(rule=centro_principal_rule)

# Función objetivo
def costo_total_rule(model):
    costo_despacho = sum(model.Precio[i] * model.X[i] for i in model.I)
    costo_transporte = sum(model.cTransp[i, model.CentroPrincipal] * model.Y[i, model.CentroPrincipal] for i in model.I if i != model.CentroPrincipal)
    return costo_despacho + costo_transporte
model.CostoTotal = pyo.Objective(rule=costo_total_rule, sense=pyo.minimize)

# Resolver el modelo
solver = pyo.SolverFactory('glpk')
results = solver.solve(model, tee=True)

# Imprimir resultados
print("Solución óptima:")

print("\nCentros elegidos y cantidad asignada:")
for i in model.I:
    try:
        print(f"Centro {i}: = {pyo.value(model.X[i]):.2f}")
    except ValueError:
        print(f"Centro {i}: Cantidad despachada no disponible")

print("\nCentros involucrados:")
for i in model.I:
    for j in model.I:
        try:
            if pyo.value(model.Y[i, j]) > 0:
                print(f"Centro {i}: Cantidad por despachadar = {pyo.value(model.X[i]):.2f}")
        except ValueError:
            print(f"Cantidad transportada de Centro {i} a Centro {j} no disponible")
            
print(f"\nCentro principal seleccionado: {model.CentroPrincipal.value}")

print("\nResumen de operación del centro principal:")

for i in model.I:
    if i != model.CentroPrincipal:
        try:
            cantidad_transportada = sum(model.Y[i, j].value for j in model.I)
            print(f'Centro {i}: Despachado = {model.X[i].value:.2f}, Cantidad transportada al centro principal = {cantidad_transportada:.2f}')
        except ValueError:
            print(f'Centro {i}: Despachado o cantidad transportada no disponible')
            

print(f"\nCosto total del modelo de optimización: = ${pyo.value(model.CostoTotal):,.2f}")

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmp7p8zbz2x.glpk.raw --wglp /tmp/tmpiirybh05.glpk.glp --cpxlp
 /tmp/tmpb4p9bj8m.pyomo.lp
Reading problem data from '/tmp/tmpb4p9bj8m.pyomo.lp'...
33 rows, 72 columns, 152 non-zeros
347 lines were read
Writing problem data to '/tmp/tmpiirybh05.glpk.glp'...
300 lines were written
GLPK Simplex Optimizer 5.0
33 rows, 72 columns, 152 non-zeros
Preprocessing...
17 rows, 64 columns, 128 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  2.300e+00  ratio =  2.300e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 17
      0: obj =   1.080000000e+07 inf =   4.700e+02 (1)
      3: obj =   8.460000000e+06 inf =   0.000e+00 (0)
*     6: obj =   7.580000000e+06 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.1 Mb (88900 bytes)
Writing basic solution to '/tmp/tmp7p8zbz2x.glpk.raw'...
114 lines were written
Solución óptima

In [161]:
import pyomo.environ as pyo

# Datos de entrada
datos = {
    "demanda": 600,
    "centros_acopio": [
        {"id": 0, "stock": 100, "produccion_potencial": 150, "precio": 10000, "tiempo_alistamiento": 1},
        {"id": 1, "stock": 200, "produccion_potencial": 200, "precio": 15000, "tiempo_alistamiento": 2},
        {"id": 2, "stock": 150, "produccion_potencial": 180, "precio": 12000, "tiempo_alistamiento": 1.5},
        {"id": 3, "stock": 120, "produccion_potencial": 170, "precio": 11000, "tiempo_alistamiento": 1.2},
        {"id": 4, "stock": 180, "produccion_potencial": 190, "precio": 14000, "tiempo_alistamiento": 2.1},
        {"id": 5, "stock": 160, "produccion_potencial": 160, "precio": 13000, "tiempo_alistamiento": 1.8},
        {"id": 6, "stock": 110, "produccion_potencial": 140, "precio": 16000, "tiempo_alistamiento": 1.7},
        {"id": 7, "stock": 140, "produccion_potencial": 155, "precio": 17000, "tiempo_alistamiento": 2.2},
        {"id": 8, "stock": 130, "produccion_potencial": 175, "precio": 18000, "tiempo_alistamiento": 1.9}
    ],
    "costos_transporte": [
        [0, 5, 7, 6, 8, 7, 9, 10, 11],
        [5, 0, 3, 4, 5, 6, 7, 8, 9],
        [7, 3, 0, 5, 4, 6, 8, 7, 10],
        [6, 4, 5, 0, 6, 7, 8, 9, 11],
        [8, 5, 4, 6, 0, 8, 7, 9, 10],
        [7, 6, 6, 7, 8, 0, 5, 8, 9],
        [9, 7, 8, 8, 7, 5, 0, 6, 7],
        [10, 8, 7, 9, 9, 8, 6, 0, 5],
        [11, 9, 10, 11, 10, 9, 7, 5, 0]
    ],
    "tiempos_transporte": [
        [0, 1, 2, 1.5, 2, 1.8, 2.1, 2.2, 2.5],
        [1, 0, 1, 1.2, 1.5, 1.3, 1.8, 1.9, 2.1],
        [2, 1, 0, 1.3, 1.4, 1.6, 2, 1.7, 2],
        [1.5, 1.2, 1.3, 0, 1.6, 1.5, 1.8, 2, 2.2],
        [2, 1.5, 1.4, 1.6, 0, 1.7, 2, 1.8, 2.3],
        [1.8, 1.3, 1.6, 1.5, 1.7, 0, 1.8, 1.9, 2],
        [2.1, 1.8, 2, 1.8, 2, 1.8, 0, 1.5, 1.8],
        [2.2, 1.9, 1.7, 2, 1.8, 1.9, 1.5, 0, 1.6],
        [2.5, 2.1, 2, 2.2, 2.3, 2, 1.8, 1.6, 0]
    ],
    "parametros_modelo": {
        "costo_tiempo": 2,
        "tiempo_maximo_definido": 5
    }
}

# Crear el modelo de optimización
model = pyo.ConcreteModel()

# Conjuntos
model.I = pyo.RangeSet(len(datos['centros_acopio']) - 1)  # Centros de acopio

# Parámetros
model.Demanda = pyo.Param(initialize=datos['demanda'])
model.Stock = pyo.Param(model.I, initialize={i: datos['centros_acopio'][i]['stock'] for i in model.I})
model.Pproduccion = pyo.Param(model.I, initialize={i: datos['centros_acopio'][i]['produccion_potencial'] for i in model.I})
model.Precio = pyo.Param(model.I, initialize={i: datos['centros_acopio'][i]['precio'] for i in model.I})
model.cTransp = pyo.Param(model.I, model.I, initialize={(i, j): datos['costos_transporte'][i][j] for i in model.I for j in model.I})
model.tTransp = pyo.Param(model.I, model.I, initialize={(i, j): datos['tiempos_transporte'][i][j] for i in model.I for j in model.I})
model.tAlistam = pyo.Param(model.I, initialize={i: datos['centros_acopio'][i]['tiempo_alistamiento'] for i in model.I})
model.cTiempo = pyo.Param(initialize=datos['parametros_modelo']['costo_tiempo'])
model.tMax = pyo.Param(initialize=datos['parametros_modelo']['tiempo_maximo_definido'])

# Selección automática del centro principal basado en el costo más bajo
centro_principal_id = min(model.I, key=lambda i: datos['centros_acopio'][i]['precio'])
model.CentroPrincipal = pyo.Param(initialize=centro_principal_id)

# Variables
model.X = pyo.Var(model.I, within=pyo.NonNegativeReals)  # Cantidad despachada desde cada centro
model.Y = pyo.Var(model.I, model.I, within=pyo.NonNegativeReals)  # Cantidad transportada entre centros

# Restricciones
model.DemandaSatisfecha = pyo.Constraint(expr=sum(model.X[i] for i in model.I) == model.Demanda)

model.StockDisponible = pyo.Constraint(model.I, rule=lambda model, i: model.X[i] <= model.Stock[i])

model.ProduccionPotencial = pyo.Constraint(model.I, rule=lambda model, i: model.X[i] <= model.Pproduccion[i])

model.TiempoMax = pyo.Constraint(model.I, rule=lambda model, i: model.tAlistam[i] + sum(model.tTransp[i, j] * model.Y[i, j] for j in model.I) <= model.tMax)

model.BalanceTransport = pyo.Constraint(model.I, rule=lambda model, i: sum(model.Y[j, i] for j in model.I) == model.X[i] if i != model.CentroPrincipal else pyo.Constraint.Skip)

model.CentroPrincipalManejo = pyo.Constraint(expr=sum(model.Y[i, model.CentroPrincipal] for i in model.I) == model.X[model.CentroPrincipal])

# Función objetivo
model.CostoTotal = pyo.Objective(
    rule=lambda model: sum(model.Precio[i] * model.X[i] for i in model.I) + sum(model.cTransp[i, model.CentroPrincipal] * model.Y[i, model.CentroPrincipal] for i in model.I if i != model.CentroPrincipal),
    sense=pyo.minimize
)

# Resolver el modelo
solver = pyo.SolverFactory('glpk')
results = solver.solve(model, tee=True)

# Imprimir resultados
print("Solución óptima:")

print("\nCentros elegidos y cantidad asignada:")
for i in model.I:
    try:
        print(f"Centro {i}: = {pyo.value(model.X[i]):.2f}")
    except ValueError:
        print(f"Centro {i}: Cantidad despachada no disponible")

print("\nCentros involucrados:")
for i in model.I:
    for j in model.I:
        try:
            if pyo.value(model.Y[i, j]) > 0:
                print(f"Centro {i}: Cantidad por despachadar = {pyo.value(model.X[i]):.2f}")
        except ValueError:
            print(f"Cantidad transportada de Centro {i} a Centro {j} no disponible")
            
print(f"\nCentro principal seleccionado: {model.CentroPrincipal.value}")

print("\nResumen de operación del centro principal:")

for i in model.I:
    if i != model.CentroPrincipal:
        try:
            cantidad_transportada = sum(model.Y[i, j].value for j in model.I)
            print(f'Centro {i}: Despachado = {model.X[i].value:.2f}, Cantidad transportada al centro principal = {cantidad_transportada:.2f}')
        except ValueError:
            print(f'Centro {i}: Despachado o cantidad transportada no disponible')
            

print(f"\nCosto total del modelo de optimización: = ${pyo.value(model.CostoTotal):,.2f}")


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmp3a5u928c.glpk.raw --wglp /tmp/tmprgc1r4j0.glpk.glp --cpxlp
 /tmp/tmp_tp6nvk7.pyomo.lp
Reading problem data from '/tmp/tmp_tp6nvk7.pyomo.lp'...
33 rows, 72 columns, 152 non-zeros
347 lines were read
Writing problem data to '/tmp/tmprgc1r4j0.glpk.glp'...
300 lines were written
GLPK Simplex Optimizer 5.0
33 rows, 72 columns, 152 non-zeros
Preprocessing...
17 rows, 64 columns, 128 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  2.300e+00  ratio =  2.300e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 17
      0: obj =   1.080000000e+07 inf =   4.700e+02 (1)
      3: obj =   8.460000000e+06 inf =   0.000e+00 (0)
*     6: obj =   7.580000000e+06 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.1 Mb (88900 bytes)
Writing basic solution to '/tmp/tmp3a5u928c.glpk.raw'...
114 lines were written
Solución óptima